In [2]:
import pandas as pd
import numpy as np
import pgmpy

print('Libraries imported')

Libraries imported


Loading Airbnb database

In [38]:
listings = pd.read_csv("listings.csv")

listings.head()

,id,listing_url,scrape_id,last_scraped,name,description,neighborhood_overview,picture_url,host_id,host_url,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,42196,https://www.airbnb.com/rooms/42196,20211217003510,2021-12-17,50 sm Studio in the historic centre,<b>The space</b><br />Really cozy and typical ...,NaN,https://a0.muscache.com/pictures/4775726/68044...,184487,https://www.airbnb.com/users/show/184487,...,4.85,4.85,4.70,NaN,t,1,1,0,0,1.32
1,46352,https://www.airbnb.com/rooms/46352,20211217003510,2021-12-17,A room in Pasolini's house,"Simple, cozy and silent room in a lived house ...",In the very nearby you have Via Saragozza whic...,https://a0.muscache.com/pictures/9f94acc3-98fc...,467810,https://www.airbnb.com/users/show/467810,...,4.79,4.80,4.59,NaN,f,2,0,2,0,2.20
2,59697,https://www.airbnb.com/rooms/59697,20211217003510,2021-12-17,COZY LARGE BEDROOM in the city center,"Cozy, spacious and bright double bedroom, in a...",NaN,https://a0.muscache.com/pictures/ac0528c4-b26f...,286688,https://www.airbnb.com/users/show/286688,...,4.79,4.82,4.66,NaN,f,2,0,2,0,2.18
3,85368,https://www.airbnb.com/rooms/85368,20211217003510,2021-12-17,Garden House Bologna,<b>The space</b><br />Garden House Bologna let...,NaN,https://a0.muscache.com/pictures/585815/331d45...,467675,https://www.airbnb.com/users/show/467675,...,5.00,4.74,4.82,NaN,f,1,1,0,0,0.34
4,145779,https://www.airbnb.com/rooms/145779,20211217003510,2021-12-17,SINGLE ROOM,<b>The space</b><br />&lt;3,NaN,https://a0.muscache.com/pictures/76537876/27c6...,705535,https://www.airbnb.com/users/show/705535,...,4.92,5.00,4.86,NaN,f,9,2,7,0,0.55


In [39]:
listings = pd.concat([listings, listings['amenities'].str.get_dummies(sep='", "').astype(bool)], axis=1)

In [44]:
columns = ['id', 'neighbourhood_cleansed', 'room_type', 'accommodates', 'bathrooms_text', 'price', 'Kitchen', 'Wifi']
listings_selection = listings[columns]

listings_selection.head()





,id,neighbourhood_cleansed,room_type,accommodates,bathrooms_text,price,Kitchen,Wifi
0,42196,Santo Stefano,Entire home/apt,2,1 bath,$68.00,True,False
1,46352,Porto - Saragozza,Private room,2,1 shared bath,$29.00,True,False
2,59697,Santo Stefano,Private room,2,1 shared bath,$50.00,True,False
3,85368,Santo Stefano,Entire home/apt,2,2 baths,$126.00,True,True
4,145779,Porto - Saragozza,Private room,1,1 shared bath,$50.00,False,False


In [45]:
listings_selection['price'] = listings_selection['price'].str.replace('[$|,]', '').astype(np.float)


listings_selection.head()

C:\Users\giach\AppData\Local\Temp/ipykernel_10356/1063439030.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  listings_selection['price'] = listings_selection['price'].str.replace('[$|,]', '').astype(np.float)
C:\Users\giach\AppData\Local\Temp/ipykernel_10356/1063439030.py:1: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  listings_selection['price'] = listings_selection['price'].str.replace('[$|,]', '').astype(np.float)
C:\Users\giach\AppData\Local\Temp/ipykernel_10356/1063439030.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_inde

,id,neighbourhood_cleansed,room_type,accommodates,bathrooms_text,price,Kitchen,Wifi
0,42196,Santo Stefano,Entire home/apt,2,1 bath,68.0,True,False
1,46352,Porto - Saragozza,Private room,2,1 shared bath,29.0,True,False
2,59697,Santo Stefano,Private room,2,1 shared bath,50.0,True,False
3,85368,Santo Stefano,Entire home/apt,2,2 baths,126.0,True,True
4,145779,Porto - Saragozza,Private room,1,1 shared bath,50.0,False,False


In [46]:
listings_selection['bathrooms_text'] = listings_selection['bathrooms_text'].str.contains('shared')

listings_selection.head()

C:\Users\giach\AppData\Local\Temp/ipykernel_10356/4129026893.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_selection['bathrooms_text'] = listings_selection['bathrooms_text'].str.contains('shared')


,id,neighbourhood_cleansed,room_type,accommodates,bathrooms_text,price,Kitchen,Wifi
0,42196,Santo Stefano,Entire home/apt,2,False,68.0,True,False
1,46352,Porto - Saragozza,Private room,2,True,29.0,True,False
2,59697,Santo Stefano,Private room,2,True,50.0,True,False
3,85368,Santo Stefano,Entire home/apt,2,False,126.0,True,True
4,145779,Porto - Saragozza,Private room,1,True,50.0,False,False


In [65]:
cat, bins = pd.cut(listings_selection['price'], [0, 50, 150, 500, 10000], labels=['<50', '50-150', '150-500', '>500'], retbins=True)

In [84]:
listings_selection['price'] = cat

C:\Users\giach\AppData\Local\Temp/ipykernel_10356/254853864.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  listings_selection['price'] = cat


In [85]:
listings_selection.describe()

,id,accommodates
count,3.453000e+03,3453.000000
mean,2.950218e+07,3.312192
std,1.523988e+07,1.644671
min,4.219600e+04,1.000000
25%,1.748597e+07,2.000000
50%,3.078707e+07,3.000000
75%,4.220094e+07,4.000000
max,5.385496e+07,16.000000


In [81]:
cat.describe()

count       3453
unique         4
top       50-150
freq        2242
Name: price, dtype: object

In [82]:

cat.groupby(by='price').value_counts()

KeyError: 'price'